In [1]:
import pandas as pd
import glob


In [2]:
df = pd.read_csv('/work/bbc6523/diverse_voices/speakers_all.csv')

In [3]:
df.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,24.0,12.0,"koussi, senegal",balanta,balanta,male,788,senegal,True,NaN,NaN,NaN
1,18.0,10.0,"buea, cameroon",cameroon,cameroon,male,1953,cameroon,True,NaN,NaN,NaN
2,48.0,8.0,"hong, adamawa, nigeria",fulfulde,fulfulde,male,1037,nigeria,True,NaN,NaN,NaN
3,42.0,42.0,"port-au-prince, haiti",haitian,haitian,male,1165,haiti,True,NaN,NaN,NaN
4,40.0,35.0,"port-au-prince, haiti",haitian,haitian,male,1166,haiti,True,NaN,NaN,NaN


In [25]:
all_recs = glob.glob('/work/bbc6523/diverse_voices/recordings/*')

In [9]:
subset = df[df['file_missing?'] ==False]

In [26]:
def check_rec_exists(row):
    string_check = '/work/bbc6523/diverse_voices/recordings/' + row['filename'] + '.mp3'
    if string_check in all_recs:
        return True
    else:
        return False
    
subset['file_found'] = subset.apply(lambda row: check_rec_exists(row), axis=1)

/tmp/ipykernel_1617164/3489302405.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['file_found'] = subset.apply(lambda row: check_rec_exists(row), axis=1)


In [7]:
subset.at[1372, 'country'] = 'laos'
subset = subset[(subset['file_found']) & (~subset['country'].isnull())]

NameError: name 'subset' is not defined

In [5]:
subset = subset[(subset['file_found']) & (~subset['country'].isnull())]

NameError: name 'subset' is not defined

In [8]:
def extract_city(row):
    loc_list = row['birthplace'].split(', ')
    return loc_list[0]
    
subset['city'] = subset.apply(lambda row: extract_city(row), axis=1)

def extract_state(row):
    loc_list = row['birthplace'].split(', ')
    if len(loc_list) > 2:
        return loc_list[1]
    else: 
        return 'none'

subset['state'] = subset.apply(lambda row: extract_state(row), axis=1)

NameError: name 'subset' is not defined

In [4]:
import plotly.express as px

grouped = subset.groupby('native_language').count().reset_index()

fig = px.bar(grouped, x='native_language', y='speakerid')
fig.show()

NameError: name 'subset' is not defined

In [59]:
natives = subset[subset['native_language'] == 'english']

grouped = natives.groupby('country').count().reset_index()

fig = px.bar(grouped, x='country', y='speakerid')
fig.show()

In [60]:
usa_natives = subset[(subset['native_language'] == 'english') & (subset['country'] == 'usa')]

grouped = usa_natives.groupby('state').count().reset_index()

fig = px.bar(grouped, x='state', y='speakerid')
fig.show()

In [80]:
subset.to_csv('english_dataset.csv', index=False)

In [63]:
from datasets import load_dataset

ds = load_dataset("distil-whisper/librispeech_long")

Generating validation split: 100%|██████████| 1/1 [00:00<00:00, 35.75 examples/s]


In [78]:
len(ds['validation']['audio'][0]['array'])

999280

In [73]:
import librosa
# Specify the file path
file_path = '/work/bbc6523/diverse_voices/recordings/zulu1.mp3'

# librosa.load returns:
# - y: a NumPy array containing the audio samples
# - sr: the sampling rate (default = 22050 Hz if you don't specify otherwise)
y, sr = librosa.load(file_path, sr=16000)  # e.g., load and resample to 16kHz

print("Audio array shape:", y.shape)
print("Sample rate:", sr)

Audio array shape: (405760,)
Sample rate: 16000


In [79]:
len(y)

405760

In [6]:
from datasets import load_dataset
from torch.utils.data.sampler import BatchSampler, RandomSampler

list_of_dfs = []
for config in ['irish_male', 'midlands_female', 'midlands_male', 'northern_female', 'northern_male', 'scottish_female', 'scottish_male', 'southern_female', 'southern_male', 'welsh_female', 'welsh_male']:

    dataset =load_dataset("ylacombe/english_dialects", config, split="train", cache_dir="/work/bbc6523/cache_dir")
    df = dataset.to_pandas()
    df['speaker_origin'] = config
    list_of_dfs.append(df)


Generating train split: 100%|██████████| 1650/1650 [00:02<00:00, 571.23 examples/s]


In [16]:

dataset =load_dataset("ylacombe/english_dialects", 'irish_male', split="train", cache_dir="/work/bbc6523/cache_dir")
df = dataset.to_pandas()
df['speaker_origin'] = config

In [22]:
df['audio'][0].keys()

dict_keys(['bytes', 'path'])

In [23]:
dataset['audio'][0].keys()

dict_keys(['path', 'array', 'sampling_rate'])

In [9]:
full_df = pd.concat(list_of_dfs)
full_df.to_csv('british_dialects.csv', index=False)

In [11]:
full_df['audio']

0       {'bytes': b'RIFF$\xa0\x06\x00WAVEfmt \x10\x00\...
1       {'bytes': b'RIFF$\x80\x08\x00WAVEfmt \x10\x00\...
2       {'bytes': b'RIFF$@\x05\x00WAVEfmt \x10\x00\x00...
3       {'bytes': b'RIFF$ \x07\x00WAVEfmt \x10\x00\x00...
4       {'bytes': b'RIFF$\x80\t\x00WAVEfmt \x10\x00\x0...
                              ...                        
1645    {'bytes': b'RIFF$\xe0\x04\x00WAVEfmt \x10\x00\...
1646    {'bytes': b'RIFF$\x00\x05\x00WAVEfmt \x10\x00\...
1647    {'bytes': b'RIFF$`\x06\x00WAVEfmt \x10\x00\x00...
1648    {'bytes': b'RIFF$`\t\x00WAVEfmt \x10\x00\x00\x...
1649    {'bytes': b'RIFF$@\x05\x00WAVEfmt \x10\x00\x00...
Name: audio, Length: 17877, dtype: object

In [15]:
df['audio'][0].keys()

dict_keys(['bytes', 'path'])